## Библиотеки

In [3]:
from tqdm import tqdm
import pickle
import os

import pandas as pd
import numpy as np
import scipy.stats as sps

from sklearn.preprocessing import StandardScaler
import plotly_express as px

import statsmodels.formula.api as smf
import pyblp

np.random.seed = 42
USE_PYBLP_OPTIONS = False

if USE_PYBLP_OPTIONS:
    pyblp.options.verbose_tracebacks = True
    pyblp.options.dtype = np.longdouble
    pyblp.options.drop_product_fields = True

## Загружаем рыночные и демографические данные

In [2]:
product_data = pd.read_parquet("data_blp/product_data.pq").reset_index(drop=True)
product_data["product_ids"] = product_data.product_ids.astype(str)
product_data.head()

,market_ids,product_ids,firm_ids,shares,prices,plant,fat_content,lactose_free,ultrapasteurized
0,adm,0,Простоквашино,0.000862,75.0,0,2.50,0,0
1,adm,1,Самокат,0.000345,83.0,0,2.50,0,0
2,adm,2,Простоквашино,0.000293,89.0,0,3.95,0,0
3,adm,3,Пискарёвское,0.000227,69.0,0,2.50,0,0
4,adm,4,Простоквашино,0.000215,123.0,0,1.50,1,1


In [3]:
agent_data = pd.read_parquet("data_blp/agent_data.pq")
agent_data = agent_data[agent_data.market_ids != 'krt'].reset_index(drop=True)
agent_data["age_squared"] = agent_data.age ** 2
agent_data = agent_data[['market_ids', 'weights', 'age', 'age_squared', 'woman', 'child', 'retiree']]

scaler = StandardScaler()
agent_data[['age', 'age_squared', 'woman', 'child', 'retiree']] = pd.DataFrame(
    scaler.fit_transform(agent_data[['age', 'age_squared', 'woman', 'child', 'retiree']]),
    columns=['age', 'age_squared', 'woman', 'child', 'retiree']
)

agent_data.head()

,market_ids,weights,age,age_squared,woman,child,retiree
0,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986
1,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986
2,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986
3,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986
4,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986


## Формулируем модель

In [4]:
delta_formulation = pyblp.Formulation('0 + prices', absorb="product_ids")
mu_formulation = pyblp.Formulation('1 + prices + plant + fat_content + lactose_free + ultrapasteurized')
agent_formulation = pyblp.Formulation('0 + age + age_squared + woman + child + retiree')

characteristics_number = len(str(mu_formulation).split(' + '))
demographics_number = len(str(agent_formulation).split(' + '))

initial_sigma = np.ones((characteristics_number, characteristics_number))
initial_pi = np.ones((characteristics_number, demographics_number))

In [5]:
nodes_df = pd.DataFrame(
    np.random.multivariate_normal(
        mean=np.zeros(characteristics_number),
        cov=np.eye(characteristics_number, characteristics_number),
        size=len(agent_data)
    )
)
nodes_columns = [f"nodes{i}" for i in range(characteristics_number)]
nodes_df.columns = nodes_columns
agent_data = pd.concat([agent_data, nodes_df], axis=1)

agent_data.head()

,market_ids,weights,age,age_squared,woman,child,retiree,nodes0,nodes1,nodes2,nodes3,nodes4,nodes5
0,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986,-0.528228,-1.036730,-1.004004,0.347946,0.823094,-0.057897
1,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986,0.116857,0.470034,-1.799174,-1.048171,0.440680,0.010141
2,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986,0.034839,0.626417,-0.794394,-0.345908,0.872745,0.521362
3,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986,-0.900022,-0.310351,-1.028148,-1.179893,0.621439,-0.222865
4,adm,0.000675,-1.856755,-1.197859,-1.110405,2.187671,-0.524986,0.203235,0.817731,0.976294,-2.036525,-1.053823,0.032350


## Конструируем differentiation instruments

### Конструируем инструментированную цену

In [6]:
price_regression = smf.ols(
    formula=f"""
    prices ~ 0 + C(market_ids) + C(firm_ids) + plant + fat_content + lactose_free + ultrapasteurized
    """, 
    data=product_data
).fit(cov_type="cluster", cov_kwds={"groups": product_data["firm_ids"]})
price_regression._results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 prices   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 10 Apr 2023   Prob (F-statistic):                nan
Time:                        18:40:40   Log-Likelihood:                -3983.4
No. Observations:                 803   AIC:                             8037.
Df Residuals:                     768   BIC:                             8201.
Df Model:                          34                                         
Covariance Type:              cluster                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
C(market_ids)[adm]                232.9919      6.165     37.792      0.000     220.909     245.075
C(market_ids)[ctr]                235.3454      5.971     39.418      0.000     223.643     247.047
C(market_ids)[frz]                230.5976      6.183     37.298      0.000     218.480     242.715
C(market_ids)[kal]                234.5413      5.497     42.666      0.000     223.767     245.315
C(market_ids)[klp]                232.1428      7.038     32.985      0.000     218.349     245.937
C(market_ids)[krgv]               234.7496      5.422     43.300      0.000     224.124     245.376
C(market_ids)[krns]               233.3613      5.247     44.472      0.000     223.077     243.646
C(market_ids)[krs]                234.4414      4.565     51.361      0.000     225.495     243.388
C(market_ids)[krv]                227.7164      8.310     27.403      0.000     211.429     244.004
C(market_ids)[msk]                234.4169      5.592     41.918      0.000     223.456     245.377
C(market_ids)[nvs]                234.2503      5.612     41.739      0.000     223.250     245.250
C(market_ids)[pdr]                229.0569      5.821     39.352      0.000     217.648     240.465
C(market_ids)[pgr]                226.3619      9.051     25.011      0.000     208.623     244.101
C(market_ids)[prm]                234.4588      5.569     42.102      0.000     223.544     245.374
C(market_ids)[psk]                235.4843      6.205     37.952      0.000     223.323     247.645
C(market_ids)[vas]                225.0830     10.846     20.753      0.000     203.825     246.341
C(market_ids)[vyb]                234.1848      5.597     41.839      0.000     223.214     245.155
C(firm_ids)[T.Aroy-D]             -32.5347      0.682    -47.738      0.000     -33.870     -31.199
C(firm_ids)[T.Nemoloko]          -120.8635      1.733    -69.747      0.000    -124.260    -117.467
C(firm_ids)[T.Parmalat]          -127.5171     11.721    -10.879      0.000    -150.490    -104.544
C(firm_ids)[T.Take A Bite]        234.4389      1.737    134.999      0.000     231.035     237.843
C(firm_ids)[T.The Gentle Nut]      -8.3593      1.730     -4.833      0.000     -11.749      -4.969
C(firm_ids)[T.Viola]              -99.8948     15.510     -6.441      0.000    -130.294     -69.495
C(firm_ids)[T.Весёлый молочник]  -112.9450      5.681    -19.880      0.000    -124.080    -101.810
C(firm_ids)[T.Вологодское]       -141.5542     15.673     -9.032      0.000    -172.273    -110.836
C(firm_ids)[T.Домик в деревне]   -129.2830     15.540     -8.319      0.000    -159.741     -98.825
C(firm_ids)[T.Пискарёвское]      -157.8993      0.509   -310.124      0.000    -158.897    -156.901
C(firm_ids)[T.Простоквашино]     -147.6888      7.728    -19.111      0.000    -162

In [7]:
product_data["instrumented_price"] = price_regression.predict(product_data)
product_data.head()

,market_ids,product_ids,firm_ids,shares,prices,plant,fat_content,lactose_free,ultrapasteurized,instrumented_price
0,adm,0,Простоквашино,0.000862,75.0,0,2.50,0,0,81.568427
1,adm,1,Самокат,0.000345,83.0,0,2.50,0,0,139.615716
2,adm,2,Простоквашино,0.000293,89.0,0,3.95,0,0,79.402290
3,adm,3,Пискарёвское,0.000227,69.0,0,2.50,0,0,71.357900
4,adm,4,Простоквашино,0.000215,123.0,0,1.50,1,1,140.029473


### Конструируем непосредственно differentiation instruments

In [8]:
price_difference_formulation = pyblp.Formulation('instrumented_price')
price_difference_instruments = pd.DataFrame(
    pyblp.build_differentiation_instruments(
        formulation=price_difference_formulation,
        product_data=product_data,
        version='quadratic'
    )
)
valid_instruments = pd.DataFrame(price_difference_instruments).apply(sum).apply(bool)
price_difference_instruments = price_difference_instruments.loc[:, price_difference_instruments.columns[valid_instruments]]
price_difference_instruments.columns = [f"demand_instruments{i}" for i in range(price_difference_instruments.shape[1])]
product_data = pd.concat([product_data, price_difference_instruments], axis=1)
product_data

,market_ids,product_ids,firm_ids,shares,prices,plant,fat_content,lactose_free,ultrapasteurized,instrumented_price,demand_instruments0,demand_instruments1
0,adm,0,Простоквашино,8.623498e-04,75.0,0,2.50,0,0,81.568427,3422.386130,416707.146685
1,adm,1,Самокат,3.445678e-04,83.0,0,2.50,0,0,139.615716,4902.736049,191054.101921
2,adm,2,Простоквашино,2.933163e-04,89.0,0,3.95,0,0,79.402290,3680.347447,430553.522523
3,adm,3,Пискарёвское,2.266105e-04,69.0,0,2.50,0,0,71.357900,0.000000,490227.262142
4,adm,4,Простоквашино,2.146623e-04,123.0,0,1.50,1,1,140.029473,7093.349285,188329.902419
...,...,...,...,...,...,...,...,...,...,...,...,...
798,vyb,49,Aroy-D,1.301313e-06,271.0,1,0.00,1,0,219.927241,0.000000,407413.513243
799,vyb,54,Viola,1.530956e-07,129.0,0,1.50,0,1,148.784797,6.449608,220188.474634
800,vyb,55,Свитлогорье,1.530956e-07,124.0,0,3.20,0,1,114.683393,1.093532,309900.662236
801,vyb,57,Alpro,6.123826e-07,259.0,1,0.00,1,0,252.461898,3.771570,661682.955859


In [9]:
characteristics_difference_formulation = pyblp.Formulation('plant + fat_content + lactose_free + ultrapasteurized')
characteristics_difference_instruments = pd.DataFrame(
    pyblp.build_differentiation_instruments(
        formulation=characteristics_difference_formulation,
        product_data=product_data,
        version='quadratic',
        interact=True
    )
)
valid_instruments = pd.DataFrame(characteristics_difference_instruments).apply(sum).apply(bool)
characteristics_difference_instruments = characteristics_difference_instruments.loc[:, characteristics_difference_instruments.columns[valid_instruments]]
characteristics_difference_instruments.columns = [
    f"demand_instruments{i + price_difference_instruments.shape[1]}" 
    for i in range(characteristics_difference_instruments.shape[1])
]

product_data = pd.concat([product_data, pd.DataFrame(characteristics_difference_instruments)], axis=1)
product_data.head()

,market_ids,product_ids,firm_ids,shares,prices,plant,fat_content,lactose_free,ultrapasteurized,instrumented_price,...,demand_instruments11,demand_instruments12,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19,demand_instruments20
0,adm,0,Простоквашино,0.000862,75.0,0,2.50,0,0,81.568427,...,20.0,-44.0,20.0,0.0,141.1800,-43.00,2.4,21.0,0.0,8.0
1,adm,1,Самокат,0.000345,83.0,0,2.50,0,0,139.615716,...,10.0,-19.0,10.0,0.0,55.9925,-19.00,1.4,12.0,1.0,9.0
2,adm,2,Простоквашино,0.000293,89.0,0,3.95,0,0,79.402290,...,20.0,-73.0,20.0,0.0,345.1225,-73.45,-9.2,21.0,0.0,8.0
3,adm,3,Пискарёвское,0.000227,69.0,0,2.50,0,0,71.357900,...,20.0,-44.0,20.0,0.0,144.2825,-44.00,1.4,22.0,1.0,9.0
4,adm,4,Простоквашино,0.000215,123.0,0,1.50,1,1,140.029473,...,20.0,-24.0,0.0,-20.0,100.9800,-22.40,10.0,20.0,12.0,33.0


In [10]:
numerical_columns = ['plant', 'fat_content', 'lactose_free', 'ultrapasteurized', 'instrumented_price'] 
numerical_columns += [
    f"demand_instruments{i}" for i in range(
        characteristics_difference_instruments.shape[1] + price_difference_instruments.shape[1]
    )
]
px.imshow(product_data[numerical_columns].corr())

In [11]:
numerical_columns = ['plant', 'fat_content', 'lactose_free', 'ultrapasteurized', 'instrumented_price'] 
numerical_columns += [
    f"demand_instruments{i}" for i in range(
        characteristics_difference_instruments.shape[1] + price_difference_instruments.shape[1]
    ) if i not in [4, 8]
]
product_data = product_data.drop(columns=["demand_instruments4", "demand_instruments8"])
px.imshow(product_data[numerical_columns].corr())

In [12]:
all_instruments = pd.concat([
    price_difference_instruments, 
    characteristics_difference_instruments.drop(columns=["demand_instruments4", "demand_instruments8"])
], axis=1)
all_instruments.columns = [
    f"demand_instruments{i}" for i in range(
        all_instruments.shape[1]
    )
]

In [13]:
product_data = pd.read_parquet("data_blp/product_data.pq").reset_index(drop=True)
product_data["product_ids"] = product_data.product_ids.astype(str)
product_data = pd.concat([product_data, all_instruments], axis=1)
product_data.head()

,market_ids,product_ids,firm_ids,shares,prices,plant,fat_content,lactose_free,ultrapasteurized,demand_instruments0,...,demand_instruments9,demand_instruments10,demand_instruments11,demand_instruments12,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18
0,adm,0,Простоквашино,0.000862,75.0,0,2.50,0,0,3422.386130,...,20.0,-44.0,20.0,0.0,141.1800,-43.00,2.4,21.0,0.0,8.0
1,adm,1,Самокат,0.000345,83.0,0,2.50,0,0,4902.736049,...,10.0,-19.0,10.0,0.0,55.9925,-19.00,1.4,12.0,1.0,9.0
2,adm,2,Простоквашино,0.000293,89.0,0,3.95,0,0,3680.347447,...,20.0,-73.0,20.0,0.0,345.1225,-73.45,-9.2,21.0,0.0,8.0
3,adm,3,Пискарёвское,0.000227,69.0,0,2.50,0,0,0.000000,...,20.0,-44.0,20.0,0.0,144.2825,-44.00,1.4,22.0,1.0,9.0
4,adm,4,Простоквашино,0.000215,123.0,0,1.50,1,1,7093.349285,...,20.0,-24.0,0.0,-20.0,100.9800,-22.40,10.0,20.0,12.0,33.0


## Оцениваем модель с differentiation instruments

In [14]:
problem = pyblp.Problem(
    product_formulations = (delta_formulation, mu_formulation),
    product_data = product_data,
    agent_formulation = agent_formulation,
    agent_data = agent_data
)

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F     I     K1    K2    D    MD    ED 
---  ---  ---  -----  ----  ----  ---  ----  ----
17   803  15   52212   1     6     5    19    1  

Formulations:
       Column Indices:           0          1         2         3            4               5        
-----------------------------  ------  -----------  -----  -----------  ------------  ----------------
 X1: Linear Characteristics    prices                                                                 
X2: Nonlinear Characteristics    1       prices     plant  fat_content  lactose_free  ultrapasteurized
       d: Demographics          age    age_squared  woman     child       retiree                     


In [15]:
with pyblp.parallel(processes=6, use_pathos=True):
    solution = problem.solve(
        sigma = initial_sigma,
        pi = initial_pi
    )
solution.to_pickle("results_1d.pkl")

Starting a pool of 6 processes ...
Started the process pool after 00:00:00.
Solving the problem ...

Nonlinear Coefficient Initial Values:
     Sigma:             1           prices          plant       fat_content   lactose_free   ultrapasteurized  |   Sigma Squared:         1           prices          plant       fat_content   lactose_free   ultrapasteurized  |        Pi:              age        age_squared       woman          child         retiree   
----------------  -------------  -------------  -------------  -------------  -------------  ----------------  |  ----------------  -------------  -------------  -------------  -------------  -------------  ----------------  |  ----------------  -------------  -------------  -------------  -------------  -------------
       1          +1.000000E+00                                                                                |         1          +1.000000E+00  +1.000000E+00  +1.000000E+00  +1.000000E+00  +1.000000E+00   +1.000000E+00

## Конструируем оптимальные инструменты

In [43]:
method = 'empirical'

if os.path.exists(f"optimal_instruments_1{method[0]}.pkl"):
    with open(f"optimal_instruments_1{method[0]}.pkl", "rb") as f:
        optimal_instruments = pickle.load(f)
else:
    optimal_instruments = solution.compute_optimal_instruments(
        method = method,
        draws = 1000,
        seed = 42
    )
    optimal_instruments.to_pickle(f"optimal_instruments_1{method[0]}.pkl")

In [44]:
product_data = pd.read_parquet("data_blp/product_data.pq").reset_index(drop=True)
product_data["product_ids"] = product_data.product_ids.astype(str)

optinstr_df = pd.DataFrame(optimal_instruments.demand_instruments)
optinstr_df.columns = map(str, range(optinstr_df.shape[1]))

product_data = pd.concat([product_data, optinstr_df], axis=1)
product_data.head()

,market_ids,product_ids,firm_ids,shares,prices,plant,fat_content,lactose_free,ultrapasteurized,0,...,41,42,43,44,45,46,47,48,49,50
0,adm,0,Простоквашино,0.000862,75.0,0,2.50,0,0,-0.004586,...,0.000091,0.000170,9.616967e-08,-4.881324e-08,2.034099e-07,0.000118,0.000222,-5.377521e-07,2.729491e-07,-1.137407e-06
1,adm,1,Самокат,0.000345,83.0,0,2.50,0,0,-0.001440,...,0.000108,0.000201,9.522124e-07,-4.833184e-07,2.014038e-06,0.000146,0.000276,-5.536248e-07,2.810056e-07,-1.170980e-06
2,adm,2,Простоквашино,0.000293,89.0,0,3.95,0,0,-0.011477,...,0.000087,0.000163,1.668876e-07,-8.470784e-08,3.529864e-07,0.000109,0.000204,3.937888e-07,-1.998770e-07,8.329083e-07
3,adm,3,Пискарёвское,0.000227,69.0,0,2.50,0,0,-0.008300,...,0.000075,0.000140,-1.933230e-07,9.812576e-08,-4.089003e-07,0.000093,0.000175,-1.058004e-07,5.370156e-08,-2.237800e-07
4,adm,4,Простоквашино,0.000215,123.0,0,1.50,1,1,0.005937,...,-0.025788,-0.033761,-1.346224e-02,6.833084e-03,-2.847418e-02,-0.025774,-0.033739,-1.346103e-02,6.832470e-03,-2.847162e-02


In [45]:
numerical_columns = ['plant', 'fat_content', 'lactose_free', 'ultrapasteurized'] 
numerical_columns += list(optinstr_df.columns.values)
px.imshow(product_data[numerical_columns].corr())

In [46]:
correlated_instruments = product_data[numerical_columns].corr().abs().apply(lambda cell: cell > 0.99)
correlated_instruments = correlated_instruments.apply(lambda col: list(col[col].index))
correlated_instruments = correlated_instruments.apply(lambda cell: map(str, cell))
correlated_instruments = correlated_instruments.apply(lambda cell: ",".join(cell))
correlated_instruments = correlated_instruments.unique()
print(correlated_instruments)
print(correlated_instruments.shape)

['plant' 'fat_content,36,37,38,39,40' 'lactose_free'
 'ultrapasteurized,46,47,48,49,50' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9'
 '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '20' '21,22,23,24,25'
 '26,27,28,29,30' '31,32,33,34,35' '41,42,43,44,45']
(29,)


In [47]:
uncorrelated_instruments = list(map(lambda x: x.split(',')[0], correlated_instruments))
print(uncorrelated_instruments)
px.imshow(product_data[uncorrelated_instruments].corr())

['plant', 'fat_content', 'lactose_free', 'ultrapasteurized', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '26', '31', '41']


In [48]:
product_data = pd.read_parquet("data_blp/product_data.pq").reset_index(drop=True)
product_data["product_ids"] = product_data.product_ids.astype(str)

optinstr_df = optinstr_df[uncorrelated_instruments[4:]]
optinstr_df.columns = [
    f"demand_instruments{i}" for i in range(optinstr_df.shape[1])
]
product_data = pd.concat([product_data, optinstr_df], axis=1)
product_data.head()

,market_ids,product_ids,firm_ids,shares,prices,plant,fat_content,lactose_free,ultrapasteurized,demand_instruments0,...,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19,demand_instruments20,demand_instruments21,demand_instruments22,demand_instruments23,demand_instruments24
0,adm,0,Простоквашино,0.000862,75.0,0,2.50,0,0,-0.004586,...,0.001959,-0.000659,-0.001772,0.000826,-0.001071,-0.000050,-0.026802,-1.983560,0.000066,0.000091
1,adm,1,Самокат,0.000345,83.0,0,2.50,0,0,-0.001440,...,0.001231,-0.001094,-0.002438,0.000392,-0.001127,-0.000439,-0.026265,-2.148909,0.000081,0.000108
2,adm,2,Простоквашино,0.000293,89.0,0,3.95,0,0,-0.011477,...,0.003596,-0.000070,-0.000985,0.002256,-0.001474,0.001154,-0.025981,-2.295053,0.000059,0.000087
3,adm,3,Пискарёвское,0.000227,69.0,0,2.50,0,0,-0.008300,...,0.002328,-0.000320,-0.001251,0.001142,-0.001018,0.000288,-0.026914,-1.824229,0.000052,0.000075
4,adm,4,Простоквашино,0.000215,123.0,0,1.50,1,1,0.005937,...,0.009502,0.002522,-0.008178,0.001182,-0.016318,-0.008622,-0.024944,-3.111955,0.000227,-0.025788


## Оцениваем модель с оптимальными инструментами

In [49]:
optimal_instruments_problem = pyblp.Problem(
    product_formulations = (delta_formulation, mu_formulation),
    product_data = product_data,
    agent_formulation = agent_formulation,
    agent_data = agent_data
)

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F     I     K1    K2    D    MD    ED 
---  ---  ---  -----  ----  ----  ---  ----  ----
17   803  15   52212   1     6     5    25    1  

Formulations:
       Column Indices:           0          1         2         3            4               5        
-----------------------------  ------  -----------  -----  -----------  ------------  ----------------
 X1: Linear Characteristics    prices                                                                 
X2: Nonlinear Characteristics    1       prices     plant  fat_content  lactose_free  ultrapasteurized
       d: Demographics          age    age_squared  woman     child       retiree                     


In [50]:
with pyblp.parallel(processes=6, use_pathos=True):
    optimal_instruments_solution = optimal_instruments_problem.solve(
        sigma = solution.sigma,
        pi = solution.pi
    )
optimal_instruments_solution.to_pickle("results_1o.pkl")

Starting a pool of 6 processes ...
Started the process pool after 00:00:00.
Solving the problem ...

Nonlinear Coefficient Initial Values:
     Sigma:             1           prices          plant       fat_content   lactose_free   ultrapasteurized  |   Sigma Squared:         1           prices          plant       fat_content   lactose_free   ultrapasteurized  |        Pi:              age        age_squared       woman          child         retiree   
----------------  -------------  -------------  -------------  -------------  -------------  ----------------  |  ----------------  -------------  -------------  -------------  -------------  -------------  ----------------  |  ----------------  -------------  -------------  -------------  -------------  -------------
       1          +1.353960E+00                                                                                |         1          +1.833208E+00  -1.697448E-01  +6.118605E-01  +3.642750E+00  +7.088967E-01   +1.678727E+00

## Получаем производные результаты (эластичности, маркапы и т.д.)

In [13]:
instruments_used = 'differentiation'

with open(f"results_1{instruments_used[0]}.pkl", "rb") as f:
        solution = pickle.load(f)

In [28]:
px.imshow(solution.compute_elasticities(name='prices', market_id='vas'))

Computing elasticities with respect to prices ...
Finished after 00:00:00.



In [29]:
px.imshow(solution.compute_diversion_ratios(name='prices', market_id='vas'))

Computing diversion ratios with respect to prices ...
Finished after 00:00:00.



## Строим доверительные интервалы для производных результатов 

In [6]:
instruments_used = 'differentiation'

with open(f"results_1{instruments_used[0]}.pkl", "rb") as f:
        solution = pickle.load(f)

In [42]:
if os.path.exists(f"bootstrap_1{instruments_used[0]}.pkl"):
    with open(f"bootstrap_1{instruments_used[0]}.pkl", "rb") as f:
        bootstrapped_results = pickle.load(f)
else:
    bootstrapped_results = solution.bootstrap(
        draws=10000, 
        seed=42
    )
    bootstrapped_results.to_pickle(f"bootstrap_1{instruments_used[0]}.pkl")

In [36]:
elasticities_bootstrapped = bootstrapped_results.compute_elasticities(name='prices', market_id='vas')

Computing elasticities with respect to prices ...
Finished 9684 out of 10000 after 00:01:00.
Finished after 00:01:02.



In [37]:
px.histogram(elasticities_bootstrapped[:, 34, 34])

In [38]:
np.mean(elasticities_bootstrapped[:, 34, 34])

3.348444481692485

In [39]:
diversion_ratios_bootstrapped = bootstrapped_results.compute_diversion_ratios(name='prices', market_id='vas')

Computing diversion ratios with respect to prices ...
Finished 9682 out of 10000 after 00:01:00.
Finished after 00:01:02.



In [40]:
px.histogram(diversion_ratios_bootstrapped[:, 9, 52])

In [41]:
np.median(diversion_ratios_bootstrapped[:, 9, 52])

0.003252249354146982

## Regression of FE on product characteristics

In [4]:
with open("results_1d.pkl", "rb") as f:
    solution = pickle.load(f)
product_data["product_fe"] = solution.xi_fe

In [14]:
products_fe_regression = smf.ols(
    formula=f"""
    product_fe ~ 0 + C(firm_ids) + plant + fat_content + lactose_free + ultrapasteurized
    """, 
    data = product_data.drop_duplicates(subset="product_ids")
).fit(cov_type="cluster", cov_kwds={"groups": product_data.drop_duplicates(subset="product_ids")["firm_ids"]})
products_fe_regression._results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             product_fe   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 19 Apr 2023   Prob (F-statistic):                nan
Time:                        17:22:32   Log-Likelihood:                -231.85
No. Observations:                  66   AIC:                             501.7
Df Residuals:                      47   BIC:                             543.3
Df Model:                          18                                         
Covariance Type:              cluster                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
C(firm_ids)[Alpro]              -42.8810      1.684    -25.465      0.000     -46.181     -39.581
C(firm_ids)[Aroy-D]             -39.2351      1.443    -27.195      0.000     -42.063     -36.407
C(firm_ids)[Nemoloko]           -18.2101      2.169     -8.395      0.000     -22.462     -13.958
C(firm_ids)[Parmalat]           -22.6546      3.728     -6.077      0.000     -29.961     -15.348
C(firm_ids)[Take A Bite]        -70.3724      1.443    -48.777      0.000     -73.200     -67.545
C(firm_ids)[The Gentle Nut]     -42.5105      1.443    -29.465      0.000     -45.338     -39.683
C(firm_ids)[Viola]              -25.7112      3.646     -7.052      0.000     -32.857     -18.565
C(firm_ids)[Весёлый молочник]   -23.6078      2.770     -8.521      0.000     -29.038     -18.178
C(firm_ids)[Вологодское]        -25.0612      4.037     -6.208      0.000     -32.973     -17.149
C(firm_ids)[Домик в деревне]    -22.3271      3.872     -5.766      0.000     -29.916     -14.738
C(firm_ids)[Пискарёвское]       -16.7322      1.546    -10.824      0.000     -19.762     -13.702
C(firm_ids)[Простоквашино]      -24.7978      2.955     -8.392      0.000     -30.589     -19.006
C(firm_ids)[Самокат]            -25.4231      1.443    -17.621      0.000     -28.251     -22.595
C(firm_ids)[Свитлогорье]        -24.7821      3.872     -6.400      0.000     -32.371     -17.193
C(firm_ids)[ЭкоНива]            -34.9841      2.875    -12.168      0.000     -40.619     -29.349
plant                           -10.3656      2.763     -3.752      0.000     -15.781      -4.950
fat_content                      -2.3100      0.618     -3.736      0.000      -3.522      -1.098
lactose_free                     -4.1201      3.685     -1.118      0.264     -11.342       3.102
ultrapasteurized                 -2.7923      2.804     -0.996      0.319      -8.288       2.704
==============================================================================
Omnibus:                       53.583   Durbin-Watson:                   1.639
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              388.492
Skew:                          -2.103   Prob(JB):                     4.36e-85
Kurtosis:                      14.117   Cond. No.                         37.6
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

## Plain Logit

In [2]:
product_data = pd.read_parquet("data_blp/product_data.pq").reset_index(drop=True)
product_data["product_ids"] = product_data.product_ids.astype(str)

delta_formulation = pyblp.Formulation('0 + prices', absorb="product_ids")
product_data["demand_instruments0"] = product_data.prices

problem = pyblp.Problem(
    product_formulations = delta_formulation,
    product_data = product_data
)

with pyblp.parallel(processes=6, use_pathos=True):
    solution = problem.solve()
solution.to_pickle("plain_logit_results_1.pkl")

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F    K1    MD    ED 
---  ---  ---  ----  ----  ----
17   803  15    1     1     1  

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices
Starting a pool of 6 processes ...
Started the process pool after 00:00:00.
Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +1.277368E-31     0      +1.000000E+00  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 2    +7.941194E-32     0      +1.000000E+00  

C

In [3]:
px.imshow(solution.compute_elasticities(name='prices', market_id='vas'))

Computing elasticities with respect to prices ...
Finished after 00:00:00.



In [4]:
px.imshow(solution.compute_diversion_ratios(name='prices', market_id='vas'))

Computing diversion ratios with respect to prices ...
Finished after 00:00:00.



In [5]:
product_data["product_fe"] = solution.xi_fe
products_fe_regression = smf.ols(
    formula=f"""
    product_fe ~ 0 + C(firm_ids) + plant + fat_content + lactose_free + ultrapasteurized
    """, 
    data = product_data.drop_duplicates(subset="product_ids")
).fit(cov_type="cluster", cov_kwds={"groups": product_data.drop_duplicates(subset="product_ids")["firm_ids"]})
products_fe_regression._results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             product_fe   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 19 Apr 2023   Prob (F-statistic):                nan
Time:                        22:34:42   Log-Likelihood:                -115.33
No. Observations:                  66   AIC:                             268.7
Df Residuals:                      47   BIC:                             310.3
Df Model:                          18                                         
Covariance Type:              cluster                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
C(firm_ids)[Alpro]              -11.7181      0.352    -33.322      0.000     -12.407     -11.029
C(firm_ids)[Aroy-D]             -11.6820      0.301    -38.772      0.000     -12.273     -11.091
C(firm_ids)[Nemoloko]           -10.4056      0.453    -22.969      0.000     -11.293      -9.518
C(firm_ids)[Parmalat]           -11.1559      0.509    -21.920      0.000     -12.153     -10.158
C(firm_ids)[Take A Bite]        -12.2966      0.301    -40.812      0.000     -12.887     -11.706
C(firm_ids)[The Gentle Nut]     -13.5529      0.301    -44.981      0.000     -14.143     -12.962
C(firm_ids)[Viola]              -13.9086      0.611    -22.752      0.000     -15.107     -12.710
C(firm_ids)[Весёлый молочник]   -11.1449      0.427    -26.084      0.000     -11.982     -10.307
C(firm_ids)[Вологодское]        -10.8281      0.643    -16.841      0.000     -12.088      -9.568
C(firm_ids)[Домик в деревне]    -10.4612      0.628    -16.663      0.000     -11.692      -9.231
C(firm_ids)[Пискарёвское]       -10.1601      0.323    -31.473      0.000     -10.793      -9.527
C(firm_ids)[Простоквашино]      -11.1074      0.359    -30.926      0.000     -11.811     -10.403
C(firm_ids)[Самокат]            -11.2408      0.301    -37.308      0.000     -11.831     -10.650
C(firm_ids)[Свитлогорье]        -13.0844      0.628    -20.841      0.000     -14.315     -11.854
C(firm_ids)[ЭкоНива]            -17.3815      0.600    -28.948      0.000     -18.558     -16.205
plant                            -1.4556      1.490     -0.977      0.328      -4.375       1.464
fat_content                       0.4163      0.129      3.224      0.001       0.163       0.669
lactose_free                      0.1968      1.593      0.124      0.902      -2.926       3.320
ultrapasteurized                 -0.5005      0.624     -0.802      0.423      -1.723       0.722
==============================================================================
Omnibus:                        1.712   Durbin-Watson:                   1.233
Prob(Omnibus):                  0.425   Jarque-Bera (JB):                1.318
Skew:                          -0.130   Prob(JB):                        0.517
Kurtosis:                       2.359   Cond. No.                         37.6
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

## Nested Logit

In [6]:
delta_formulation = pyblp.Formulation('0 + prices', absorb="product_ids")

price_regression = smf.ols(
    formula=f"""
    prices ~ 0 + C(market_ids) + C(firm_ids) + plant + fat_content + lactose_free + ultrapasteurized
    """, 
    data=product_data
).fit(cov_type="cluster", cov_kwds={"groups": product_data["firm_ids"]})
product_data["instrumented_price"] = price_regression.predict(product_data)

problem = pyblp.Problem(
    product_formulations = delta_formulation,
    product_data = product_data
)

price_difference_formulation = pyblp.Formulation('instrumented_price')
price_difference_instruments = pd.DataFrame(
    pyblp.build_differentiation_instruments(
        formulation=price_difference_formulation,
        product_data=product_data,
        version='quadratic'
    )
)
valid_instruments = pd.DataFrame(price_difference_instruments).apply(sum).apply(bool)
price_difference_instruments = price_difference_instruments.loc[:, price_difference_instruments.columns[valid_instruments]]
price_difference_instruments.columns = [f"demand_instruments{i}" for i in range(price_difference_instruments.shape[1])]
product_data = pd.concat([product_data, price_difference_instruments], axis=1)

characteristics_difference_formulation = pyblp.Formulation('plant + fat_content + lactose_free + ultrapasteurized')
characteristics_difference_instruments = pd.DataFrame(
    pyblp.build_differentiation_instruments(
        formulation=characteristics_difference_formulation,
        product_data=product_data,
        version='quadratic',
        interact=True
    )
)
valid_instruments = pd.DataFrame(characteristics_difference_instruments).apply(sum).apply(bool)
characteristics_difference_instruments = characteristics_difference_instruments.loc[:, characteristics_difference_instruments.columns[valid_instruments]]
characteristics_difference_instruments.columns = [
    f"demand_instruments{i + price_difference_instruments.shape[1]}" 
    for i in range(characteristics_difference_instruments.shape[1])
]

product_data = pd.concat([product_data, pd.DataFrame(characteristics_difference_instruments)], axis=1)
product_data = product_data.drop(columns=["demand_instruments4", "demand_instruments8"])
all_instruments = pd.concat([
    price_difference_instruments, 
    characteristics_difference_instruments.drop(columns=["demand_instruments2", "demand_instruments3", "demand_instruments4", "demand_instruments8"])
], axis=1)
all_instruments.columns = [
    f"demand_instruments{i}" for i in range(
        all_instruments.shape[1]
    )
]

product_data = pd.read_parquet("data_blp/product_data.pq").reset_index(drop=True)
product_data["product_ids"] = product_data.product_ids.astype(str)
product_data["nesting_ids"] = product_data.plant
product_data = pd.concat([product_data, all_instruments], axis=1)

problem = pyblp.Problem(
    product_formulations = delta_formulation,
    product_data = product_data
)

with pyblp.parallel(processes=6, use_pathos=True):
    solution = problem.solve(
        rho = np.random.uniform(low=0.01, high=1, size=product_data.nesting_ids.unique().shape[0])
    )
solution.to_pickle("nested_logit_results_1.pkl")

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F    K1    MD    ED 
---  ---  ---  ----  ----  ----
17   803  15    1     1     1  

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices
Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F    K1    MD    ED    H 
---  ---  ---  ----  ----  ----  ---
17   803  15    1     17    1     2 

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices
Starting a pool of 6 processes ...
Started the process pool after 00:00:00.
Solving the problem ...

Rho Initial Values:
      0              1      
-------------  -------------
+3.641855E-01  +6.557241E-01

Rho Lower Bounds:
      0              1      
-------------  -------------
+0.000000E+00  +0.000000

In [7]:
px.imshow(solution.compute_elasticities(name='prices', market_id='vas'))

Computing elasticities with respect to prices ...
Finished after 00:00:00.



In [8]:
px.imshow(solution.compute_diversion_ratios(name='prices', market_id='vas'))

Computing diversion ratios with respect to prices ...
Finished after 00:00:00.



In [9]:
bootstrapped_results = solution.bootstrap(
    draws=10000, 
    seed=42
)

Bootstrapping results ...

Encountered a numerical error when solving for a realization of equilibrium prices and shares. Errors encountered: invalid value, overflow.
Encountered a numerical error when solving for a realization of equilibrium prices and shares. Errors encountered: overflow.

Bootstrapped results after 00:01:13.

Bootstrapped Results Summary:
Computation  Bootstrap
   Time        Draws  
-----------  ---------
 00:01:13      10000  


In [10]:
elasticities_bootstrapped = bootstrapped_results.compute_elasticities(name='prices', market_id='vas')

Computing elasticities with respect to prices ...

Encountered a numerical error when computing a post-estimation output. Errors encountered: invalid value.
Encountered a numerical error when computing a post-estimation output. Errors encountered: invalid value, overflow.

Finished after 00:00:07.



In [11]:
px.histogram(elasticities_bootstrapped[:, 34, 34])

In [12]:
diversion_ratios_bootstrapped = bootstrapped_results.compute_diversion_ratios(name='prices', market_id='vas')

Computing diversion ratios with respect to prices ...

Encountered a numerical error when computing a post-estimation output. Errors encountered: invalid value.
Encountered a numerical error when computing a post-estimation output. Errors encountered: invalid value, overflow.

Finished after 00:00:07.



In [13]:
px.histogram(diversion_ratios_bootstrapped[:, 9, 52])

In [14]:
product_data["product_fe"] = solution.xi_fe
products_fe_regression = smf.ols(
    formula=f"""
    product_fe ~ 0 + C(firm_ids) + plant + fat_content + lactose_free + ultrapasteurized
    """, 
    data = product_data.drop_duplicates(subset="product_ids")
).fit(cov_type="cluster", cov_kwds={"groups": product_data.drop_duplicates(subset="product_ids")["firm_ids"]})
products_fe_regression._results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             product_fe   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 19 Apr 2023   Prob (F-statistic):                nan
Time:                        22:36:59   Log-Likelihood:                -140.17
No. Observations:                  66   AIC:                             318.3
Df Residuals:                      47   BIC:                             359.9
Df Model:                          18                                         
Covariance Type:              cluster                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
C(firm_ids)[Alpro]                5.6170      0.461     12.191      0.000       4.714       6.520
C(firm_ids)[Aroy-D]               4.2821      0.395     10.847      0.000       3.508       5.056
C(firm_ids)[Nemoloko]             1.1439      0.594      1.927      0.054      -0.020       2.307
C(firm_ids)[Parmalat]            -0.2298      1.216     -0.189      0.850      -2.614       2.154
C(firm_ids)[Take A Bite]         10.2491      0.395     25.962      0.000       9.475      11.023
C(firm_ids)[The Gentle Nut]       3.3078      0.395      8.379      0.000       2.534       4.082
C(firm_ids)[Viola]                1.0211      1.228      0.832      0.406      -1.385       3.428
C(firm_ids)[Весёлый молочник]     0.7185      0.847      0.849      0.396      -0.941       2.378
C(firm_ids)[Вологодское]         -1.1034      1.321     -0.835      0.404      -3.693       1.486
C(firm_ids)[Домик в деревне]     -0.4775      1.282     -0.373      0.709      -2.989       2.034
C(firm_ids)[Пискарёвское]        -1.9884      0.423     -4.701      0.000      -2.817      -1.159
C(firm_ids)[Простоквашино]       -0.1544      0.925     -0.167      0.867      -1.968       1.659
C(firm_ids)[Самокат]              1.4448      0.395      3.660      0.000       0.671       2.219
C(firm_ids)[Свитлогорье]         -0.3761      1.282     -0.293      0.769      -2.888       2.136
C(firm_ids)[ЭкоНива]             -0.1328      0.787     -0.169      0.866      -1.675       1.409
plant                            -7.3019      1.173     -6.223      0.000      -9.602      -5.002
fat_content                      -0.0413      0.169     -0.244      0.807      -0.373       0.290
lactose_free                      1.4953      1.358      1.101      0.271      -1.166       4.157
ultrapasteurized                  0.7121      1.032      0.690      0.490      -1.311       2.736
==============================================================================
Omnibus:                       48.846   Durbin-Watson:                   1.437
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              318.610
Skew:                           1.894   Prob(JB):                     6.53e-70
Kurtosis:                      13.075   Cond. No.                         37.6
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""